In [1]:
import time 
import numpy as np 
import pandas as pd
import os 
import pandas_ta as ta
import ccxt
from datetime import datetime
from xgboost import XGBRegressor  
import optuna 
from tqdm.auto import tqdm
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from sklearn.preprocessing import MinMaxScaler
import sys
if not "Informer2020" in sys.path:
    sys.path += ["Informer2020"]
from Informer2020.models.model import Informer, InformerStack

import torch 
import torch.nn as nn 
import torch.nn.functional as F
from torch.utils.data import Dataset, TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [2]:
chart_df = pd.read_feather('BTC_USDT-4h.feather')
chart_df['date'] = pd.to_datetime(chart_df['date'])
chart_df['date'] = chart_df['date'].dt.tz_localize(None) 

hours, days, months = [], [], [] 
for dt in tqdm(chart_df["date"]): 
    dtobj = pd.to_datetime(dt) 
    hour = dtobj.hour 
    day = dtobj.day 
    month = dtobj.month 
    hours.append(hour) 
    days.append(day) 
    months.append(month) 

chart_df["hours"] = hours 
chart_df["days"] = days 
chart_df["months"] = months 

# define targets 
close = chart_df.close.values 
targets = [] 

for i in range(len(close) - 1): 
    targets.append(close[i+1]) 
targets.append(None) 
chart_df["targets"] = targets 

  0%|          | 0/13719 [00:00<?, ?it/s]

In [3]:
chart_df.drop(columns={"date"}, inplace=True)

In [4]:
scaler = MinMaxScaler() 
datetime_info = chart_df[["months", "days", "hours"]].values 

In [5]:
train_size = int(chart_df.shape[0] * 0.8) 

train_datetime_info = datetime_info[:train_size] 
val_datetime_info = datetime_info[train_size:] 

train_chart_df = chart_df.iloc[:train_size] 
val_chart_df = chart_df.iloc[train_size:] 

# Min-Max Scaling 
train_chart_df = scaler.fit_transform(train_chart_df[["open", "high", "low", "close", "volume"]]) 
val_chart_df = scaler.transform(val_chart_df[["open", "high", "low", "close", "volume"]]) 

In [6]:
def preprocess_data_for_informer(chart_df, datetime_info, window_size=42, target_size=1): 
    cur_enc_inputs, cur_dec_inputs, cur_targets, cur_enc_marks, cur_dec_marks = [], [], [], [], [] 
    
    for i in range(0, len(chart_df) - window_size - target_size): 
        x = chart_df[i:i+window_size, [0, 1, 2, 3, 4]] 
        y = chart_df[i+window_size:i+window_size+target_size, 3] 
        start_tokens = chart_df[i:i+window_size, 3].reshape((-1, 1)) 
        y_0 = np.zeros((target_size, 1)) 
    
        cur_enc_inputs.append(x) 
        cur_dec_inputs.append(np.concatenate([start_tokens, y_0])) 
        cur_targets.append(y) 
        cur_enc_marks.append(datetime_info[i:i+window_size]) 
        cur_dec_marks.append(datetime_info[i+window_size:i+window_size+target_size])
    
    cur_enc_inputs = torch.tensor(cur_enc_inputs).float() 
    cur_dec_inputs = torch.tensor(cur_dec_inputs).float() 
    cur_targets = torch.tensor(cur_targets).float() 
    cur_enc_marks = torch.tensor(cur_enc_marks).float() 
    cur_dec_marks = torch.tensor(cur_dec_marks).float() 
    
    return cur_enc_inputs, cur_dec_inputs, cur_targets, cur_enc_marks, cur_dec_marks 
    

In [7]:
train_enc_inputs, train_dec_inputs, train_targets, train_enc_marks, train_dec_marks = preprocess_data_for_informer(train_chart_df, train_datetime_info)

val_enc_inputs, val_dec_inputs, val_targets, val_enc_marks, val_dec_marks = preprocess_data_for_informer(val_chart_df, val_datetime_info)

train_enc_inputs.shape, train_dec_inputs.shape, train_targets.shape, train_enc_marks.shape, train_dec_marks.shape, val_enc_inputs.shape, val_dec_inputs.shape, val_targets.shape, val_enc_marks.shape, val_dec_marks.shape

/tmp/ipykernel_784/1000196690.py:16: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  cur_enc_inputs = torch.tensor(cur_enc_inputs).float()


(torch.Size([10932, 42, 5]),
 torch.Size([10932, 43, 1]),
 torch.Size([10932, 1]),
 torch.Size([10932, 42, 3]),
 torch.Size([10932, 1, 3]),
 torch.Size([2701, 42, 5]),
 torch.Size([2701, 43, 1]),
 torch.Size([2701, 1]),
 torch.Size([2701, 42, 3]),
 torch.Size([2701, 1, 3]))

In [8]:
batch_size = 128
train_data = TensorDataset(train_enc_inputs, train_dec_inputs, train_targets, train_enc_marks, train_dec_marks)
train_sampler = RandomSampler(train_data) 
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

val_data = TensorDataset(val_enc_inputs, val_dec_inputs, val_targets, val_enc_marks, val_dec_marks) 
val_sampler = SequentialSampler(val_data) 
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

In [12]:
best_val_loss = np.inf 
device = torch.device("cuda")
model = InformerStack(enc_in=5, dec_in=1, c_out=1, seq_len=42, label_len=42, out_len=1, freq="4h")  
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
loss_func = torch.nn.L1Loss()
train_losses, val_losses = [], []
epochs = 20 
model.train() 
model.zero_grad()
for epoch in tqdm(range(epochs), position=0, leave=True, desc="Epochs"):
    model.train()
    train_loss = 0
    for step, batch in tqdm(enumerate(train_dataloader), total=len(train_dataloader), desc="training"): 
        batch = (t.to(device) for t in batch) 
        enc_inputs, dec_inputs, targets, enc_marks, dec_marks = batch 
        output = model(x_enc=enc_inputs, x_mark_enc=enc_marks, x_dec=dec_inputs, x_mark_dec=dec_marks)
        loss = loss_func(output, targets) 
        train_loss += loss.item() 
        loss.backward() 
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step() 
        model.zero_grad() 
    model.eval() 
    val_loss = 0 
    for batch in tqdm(val_dataloader, total=len(val_dataloader), desc="validating"): 
        batch = (t.to(device) for t in batch) 
        enc_inputs, dec_inputs, targets, enc_marks, dec_marks = batch 
        with torch.no_grad(): 
            outputs = model(x_enc=enc_inputs, x_mark_enc=enc_marks, x_dec=dec_inputs, x_mark_dec=dec_marks) 
            loss = loss_func(outputs, targets)
            val_loss += loss.item()
    
    avg_train_loss = train_loss / len(train_dataloader) 
    avg_val_loss = val_loss / len(val_dataloader) 
    
    print(f"Epoch:{epoch} | Avg Train Loss:{avg_train_loss} | Avg Val Loss:{avg_val_loss}")
    
    if best_val_loss > avg_val_loss: 
        best_val_loss = avg_val_loss 
        torch.save(model.state_dict(), "BTC_Informer.pt")

Epochs:   0%|          | 0/20 [00:00<?, ?it/s]

training:   0%|          | 0/86 [00:00<?, ?it/s]

validating:   0%|          | 0/22 [00:00<?, ?it/s]

Epoch:0 | Avg Train Loss:0.3376868665218353 | Avg Val Loss:0.12915989553386514


/usr/local/lib/python3.9/dist-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([13, 1])) that is different to the input size (torch.Size([13, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


...

validating:   0%|          | 0/22 [00:00<?, ?it/s]

Epoch:18 | Avg Train Loss:0.19421882806129234 | Avg Val Loss:0.2423662692308426


training:   0%|          | 0/86 [00:00<?, ?it/s]

validating:   0%|          | 0/22 [00:00<?, ?it/s]

Epoch:19 | Avg Train Loss:0.19425832237615143 | Avg Val Loss:0.18662975237450816


In [15]:
# inference 
# save scaler 
import joblib 
from joblib import dump
dump(scaler, 'informer_scaler.joblib')

['informer_scaler.joblib']